# Producir dataset para entrenamiento

### Nota Importante

Los datos recopilados fueron obtenidos durante Febrero de 2020, los precios, el kilometraje y el listado de vehículos es altamente probable que hayan cambiado desde entonces por el uso que hayan dado a cada vehículo, las ventas que se hayan concretado, vehículos nuevos anunciados y/o por factores macroeconómicos que impacten a México.

## Análisis preliminar

Antes de decidir qué información utilizar para predecir el precio de un auto, hace falta analizar qué información se tiene.

In [1]:
#Importar pandas para cargar datos
import pandas as pd

In [2]:
#Leer archivo CSV.
#Eliminar columna de URL. Esta columna es exclusivamente para hacer referencia
#al anuncio original y es única para cada auto, por lo que no nos dice nada
#que pueda ayudar al algoritmo a generalizar cómo predecir el precio.
df = pd.read_csv("./carros_clean.csv", index_col=0).drop(['url'], axis=1)

In [3]:
#Vista previa de la información disponible en el DataFrame
df

,brand,model,version,year,km,price
0,Ford,Ikon,Ambiente Ac,2001,123467,31500
1,Ford,Focus,LX,2000,140000,33000
2,Ford,Sable,LS Premium,2001,145630,33000
3,Chevrolet,Chevy Sedán,1.6L Monza,2001,147500,35000
4,Ford,Focus,Ambiente,2001,219143,35000
...,...,...,...,...,...,...
12808,Suzuki,Ignis,5P GLX 1.2L TM5 A/AC. AUT. VE RA-16,2018,30000,2159200
12809,Mercedes Benz,Clase G,500 4X4,2018,11582,2299000
12810,Mercedes Benz,Maybach,S 650,2019,6807,2849900
12811,Mercedes Benz,AMG GT,S,2019,2283,2999900


### Variables numéricas Vs. Cadenas

Las columnas `year` y `km` contienen variables numéricas, lo que nos facilita mucho para un algoritmo de regresión lineal, ¿pero qué hay con las columnas `brand`, `model` y `version`? Finalmente, son las que contienen la información de qué tipo de carro es al que se hace referencia.

Es muy fácil imaginar los motivos que contribuyen a que un carro de marca Mercedes Benz o BMW será mucho más costoso que un Chevrolet, aunque sean del mismo año y tengan el mismo kilometraje, pero la regresión lineal solamente entiende de números, las letras, por su valor numérico en ASCII u otra forma de codificar las letras que forman el nombre de la marca, no significan mucho, ni se traducen en valores reales que influyen al precio del auto. Por ejemplo, el valor numérico de Ford por sus letras es menor que el de Chevrolet, sin embargo, es más común que los coches Chevrolet sean más baratos porque tienen más modelos de bajo costo (como Matiz, Chevy, o Spark), que la marca Ford. Y al mismo tiempo BMW tiene menos letras que Ford, pero es bien conocido que es una marca mucho más costosa en sus vehículos.

Habiendo dicho esto, ¿entonces qué hacemos para interpretar el significado de si un auto es de una marca u otra en forma numérica?

### Variables categóricas

La forma normal de visualizar los valores de cadena que representan *algo*, y que se repiten en una columna de nuestros datos es en forma de "categorías". Esto significa que las categorías se pueden representar con un número (e.g. Ford = 1, Chevrolet = 2, BMW = 3, etc.).

A su vez, estas variables pueden ser de dos tipos: ordinales y nominales. Las variables nominales son sólo "nombres", y las ordinales implican que hay un "orden" implícito en estas categorías. En nuestro caso, no hay un orden implícito, aunque queramos pensar que sí se pueden poner en una escala de las marcas menos costosas a las más costosas. La realidad es que como los precios cambian año con año, y cada que una marca saca un modelo distinto, necesitamos algo menos ambiguo, o menos cambiante para darle significado a nuestras categorías.

¿Y entonces? ¿Acaso no nos regresa esto que el nombre de la marca realmente no signifique nada? Pues sí, en este caso sólo tradujimos una cadena a un número de categoría, y como estamos tratando con más de dos marcas, y una variable nominal no dicotomizada (o no binaria), no nos ayuda mucho, necesitamos pensar en cómo dicotomizarla, para que cada peso que multiplique la variable que represente la marca del auto, se traduzca directamente en la influencia que tiene esta variable sobre el precio final.

### Dummy variables

Quizás la pregunta correcta no sea necesariamente `¿de qué marca es?` sino `¿es de marca 'XYZ' o no?`. Replanteando la pregunta de esta manera, podemos tener variables que son independientes una de la otra, y que contribuyen independientemente al precio final. Esto es, que separamos las marcas, para que cada una no sea afectada por las demás.

Lograr esto significa crear las llamadas `dummy variables` (o variables de paja / variables postizas). Esto se hace creando una variable por cada una de las categorías, haciéndolas dicótomas, y representadas exclusivamente por valores de `1` o `0`. En nuestro caso, si tuviéramos las 3 marcas: Kia, Ford y Chevrolet, cada una tendría su propia variable.

#### Categorías originales

|    Marca  |   Modelo  |  Precio |
|:---------:|:---------:|:-------:|
|   Kia     | Stinger   | 7049000 |
| Chevrolet |Chevy Sedán|  35000  |
| Ford      | Ikon      |  31500  |

#### Categorías dicotomizadas (dummy variables)

|  Kia  | Chevrolet | Ford |    Modelo  |  Precio |
|:-----:|:---------:|:----:|:----------:|:-------:|
| 1     | 0         |   0  |    Stinger | 7049000 |
| 0     | 1         |   0  | Chevy Sedán|  35000  |
| 0     | 0         |   1  |    Ikon    |  31500  |

Una vez que lo anterior se entiende, podemos entender cómo utilizar todas las categorías no ordinales para calcular los pesos que se relacionan con ellas, que también es el caso con el modelo y las versiones de los vehículos.

Veamos ahora cuántos valores tenemos para cada uno.

In [4]:
print("Marcas distintas:", len(df.brand.unique()))
print("Modelos distintos:", len(df.model.unique()))
print("Versiones distintas:", len(df.version.unique()))

Marcas distintas: 47
Modelos distintos: 531
Versiones distintas: 3939


47 marcas son un número considerable, 531 modelos para estas marcas significaría un número todavía mayor de variables dicótomas, que se traduciría en mayor necesidad de poder de computo para calcular los pesos en todas ellas, pero 3939 versiones para estos 531 modelos distintos, es casi como 1 de cada 4 dueños hubiera inventado su propia versión, y no representa necesariamente la versión real del vehículo. Esto es, que en lugar de ver para un modelo de Chevrolet las versiones `LS`, `LT` o `LTS` de un vehículo, vemos `Equipado AC Transmisión manual`. Si cada versión es lo que cada quién escribió con su palabra, no será tarea fácil generalizar a qué versión se refieren del vehículo, veamos qué debemos hacer al respecto.

### Encontrar valores anómalos y poco significativos

Hasta este punto sólo sabemos que todas las columnas en todas sus filas tienen un valor, pero no hemos encontrado qué otros casos de valores anómalos existen, además de lo que resalta a simple vista con las versiones, como mencionamos anteriormente. Quizás haya alguna marca que sólo tenga un vehículo, o un modelo con las letras al revés (Chvey en vez de Chevy, por ejemplo). Descartar algunos de estos, o corregirlos para agruparlos en la categoría correcta nos podría ahorras varias dimensiones, y eso nos ahorraría precioso poder de cómputo.

Empecemos por ver cuántos vehículos hay por marca, y descartar las marcas donde haya sólo un vehículo.

In [39]:
#Inicializar una lista con marcas de autos que sólo tengan un registro
unicos = []

#Revisemos el conteo de cada marca
for marca in df.brand.unique():
    cuenta = len(df[df.brand == marca])
    if cuenta == 1:
        unicos.append(marca)

print(f"Marcas con carros únicos: {unicos}")
print(f"Registros originales: {len(df)}")
df = df[~df.brand.isin(unicos)]
print(f"Registros originales: {len(df)}")

Marcas con carros únicos: ['Ferrari', 'Lamborghini', 'SRT', 'JAC']
Registros originales: 12812
Registros originales: 12808


Bueno, visto lo anterior, es entendible que no haya gente anunciando Lamborghinis y Ferraris por doquier en un sitio de venta de segunda mano.

Continuemos por realizar un ejercicio similar con las versiones, identifiquemos cuántas versiones por modelo hay, quizás podríamos concluir que las versiones no están tan alejadas entre sí y tienen muchas similitud que podríamos corregir si truncamos o corregimos parte del texto en la versión. Intentemos con una marca que no haya tenido demasiados modelos distintos a través de los años, como Mazda.

In [67]:
df[df.brand == 'Mazda'].groupby(['brand', 'model']).size()

brand  model      
Mazda  2               28
       2 Sedán          8
       3 Hatchback     95
       3 Sedan        180
       5               13
       6               60
       CX-3            16
       CX-5            66
       CX-7            27
       CX-9            53
       MX-5            19
       Speed3           2
dtype: int64

180 versiones para el mismo modelo de auto suena a demasiado (Mazda 3 Sedan)